In [2]:
#!pip install --upgrade pip setuptools wheel
#!pip install pyarrow

import pandas as pd
from statsmodels.formula.api import ols

Aggregating inflation data and merging inflation and location data.

In [3]:
inflation_data = pd.read_feather('../data/interim/inflation-data-clean.ftr')
long_lat_data = pd.read_feather('../data/interim/countries-with-long-lat-data.ftr')

inflation_data_aggregated = inflation_data.groupby('Reference area').agg({'OBS_VALUE': 'mean'}).reset_index()

print(inflation_data_aggregated.head)
print(long_lat_data.head)

inflation_data_aggregated.rename(columns={'Reference area': 'name'}, inplace=True)

merged_data = pd.merge(inflation_data_aggregated, long_lat_data, on= 'name', how= 'inner')



<bound method NDFrame.head of      Reference area  OBS_VALUE
0           Austria   2.634513
1           Belgium   2.473894
2          Bulgaria   3.686726
3           Croatia   2.717257
4           Czechia   3.402655
5           Denmark   1.823894
6           Estonia   4.445133
7           Finland   2.020354
8            France   1.902655
9           Germany   2.208407
10           Greece   1.868142
11          Hungary   4.867257
12          Iceland   4.505310
13          Ireland   1.466814
14            Italy   2.051327
15           Latvia   4.308407
16        Lithuania   4.122124
17       Luxembourg   2.352655
18      Netherlands   2.322566
19           Norway   2.623451
20           Poland   3.346018
21         Portugal   1.898673
22  Slovak Republic   3.049558
23         Slovenia   2.471239
24            Spain   2.111947
25           Sweden   2.111947
26      Switzerland   0.511504
27          Türkiye  18.028319
28   United Kingdom   2.904867>
<bound method NDFrame.head of          

Linear regressions of the inflation rate on longitude, latitude and combined regression on both.

In [4]:
lm_lat = ols("OBS_VALUE ~ latitude", data = merged_data).fit()

print(lm_lat.summary())

                            OLS Regression Results                            
Dep. Variable:              OBS_VALUE   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.9382
Date:                Wed, 13 Nov 2024   Prob (F-statistic):              0.341
Time:                        13:56:30   Log-Likelihood:                -72.182
No. Observations:                  29   AIC:                             148.4
Df Residuals:                      27   BIC:                             151.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      7.1080      4.023      1.767      0.0

In [5]:
lm_long = ols("OBS_VALUE ~ longitude", data = merged_data).fit()

print(lm_long.summary())

                            OLS Regression Results                            
Dep. Variable:              OBS_VALUE   R-squared:                       0.188
Model:                            OLS   Adj. R-squared:                  0.158
Method:                 Least Squares   F-statistic:                     6.255
Date:                Wed, 13 Nov 2024   Prob (F-statistic):             0.0188
Time:                        13:56:30   Log-Likelihood:                -69.656
No. Observations:                  29   AIC:                             143.3
Df Residuals:                      27   BIC:                             146.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.9919      0.719      2.769      0.0

In [6]:
lm_comb = ols("OBS_VALUE ~ latitude + longitude", data=merged_data).fit()

print(lm_comb.summary())

                            OLS Regression Results                            
Dep. Variable:              OBS_VALUE   R-squared:                       0.205
Model:                            OLS   Adj. R-squared:                  0.144
Method:                 Least Squares   F-statistic:                     3.362
Date:                Wed, 13 Nov 2024   Prob (F-statistic):             0.0503
Time:                        13:56:30   Log-Likelihood:                -69.342
No. Observations:                  29   AIC:                             144.7
Df Residuals:                      26   BIC:                             148.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.8369      3.838      1.260      0.2